# Fine tune a PyTorch BERT model and deploy it with Elastic Inference on Amazon SageMaker CN

Text classification is a technique for putting text into different categories and has a wide range of applications: email providers use text classification to detect to spam emails, marketing agencies use it for sentiment analysis of customer reviews, and moderators of discussion forums use it to detect inappropriate comments.

In the past, data scientists used methods such as [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), [word2vec](https://en.wikipedia.org/wiki/Word2vec), or [bag-of-words (BOW)](https://en.wikipedia.org/wiki/Bag-of-words_model) to generate features for training classification models. While these techniques have been very successful in many NLP tasks, they don't always capture the meanings of words accurately when they appear in different contexts. Recently, we see increasing interest in using Bidirectional Encoder Representations from Transformers (BERT) to achieve better results in text classification tasks, due to its ability more accurately encode the meaning of words in different contexts.

BERT was trained on BookCorpus and English Wikipedia data, which contain 800 million words and 2,500 million words, respectively. Training BERT from scratch would be prohibitively expensive. By taking advantage of transfer learning, one can quickly fine tune BERT for another use case with a relatively small amount of training data to achieve state-of-the-art results for common NLP tasks, such as text classification and question answering. 

Amazon SageMaker is a fully managed service that provides developers and data scientists with the ability to build, train, and deploy machine learning (ML) models quickly. Amazon SageMaker removes the heavy lifting from each step of the machine learning process to make it easier to develop high-quality models. The SageMaker Python SDK provides open source APIs and containers that make it easy to train and deploy models in Amazon SageMaker with several different machine learning and deep learning frameworks.

Our customers often ask for quick fine-tuning and easy deployment of their NLP models. Furthermore, customers prefer low inference latency and low model inference cost. [Amazon Elastic Inference](https://aws.amazon.com/machine-learning/elastic-inference) enables attaching GPU-powered inference acceleration to endpoints, reducing the cost of deep learning inference without sacrificing performance.

This blog post demonstrates how to use Amazon SageMaker to fine tune a PyTorch BERT model and deploy it with Elastic Inference. This work is inspired by a post by [Chris McCormick and Nick Ryan](https://mccormickml.com/2019/07/22/BERT-fine-tuning).

In this example, we walk through our dataset, the training process, and finally model deployment. 

# Setup

To start, we import some Python libraries and initialize a SageMaker session, S3 bucket and prefix, and IAM role.

In [1]:
# need torch 1.3.1 for elastic inference
!pip install torch==1.3.1
!pip install transformers

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import numpy as np
import pandas as pd
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-bert-cn"

role = sagemaker.get_execution_role()

# Prepare training data

We use Corpus of Linguistic Acceptability (CoLA) (https://nyu-mll.github.io/CoLA/), a dataset of 10,657 English sentences labeled as grammatical or ungrammatical from published linguistics literature. We download and unzip the data using the following code:

### Get sentences and labels

Let us take a quick look at our data. First we read in the training data. The only two columns we need are the sentence itself and its label. 

In [11]:
df = pd.read_csv("./data/train.tsv",delimiter="\t")
sentences = df.text_a.values
labels = df.label.values

Printing out a few sentences shows us how sentences are labeled based on their grammatical completeness. 

In [12]:
print(sentences[20:25])
print(labels[20:25])

['非常不错，服务很好，位于市中心区，交通方便，不过价格也高！'
 '两岁的儿子特别喜欢车,尤其是火车，于是在朋友推荐下我买了一套。没想到孩子特别喜欢，尤其是第一本《轱辘轱辘转》，天天睡觉前都要把书抱到床上看。他会不停的问我“这是什么车？”，“这是什么车？”。现在已经把第一本书都翻烂了。走在路上，看到车时，他会非常兴奋的告诉我“妈妈，水泥搅拌车！”。书中各种各样的车，给了孩子很多想像的空间，而且这是我第一次在当当上买书就非常的成功，不错，下次还来。'
 '书看到最后几页居然是?双眼皮?,所有的字都印成了两遍头，根本就不能看。书的质量实在太差.'
 '有些东西不赞同，事后的捷径，不过如此，年青人该经历的还是要去体验，否则拥有后还会抛弃，因为那不是他自己选择过的，所以此书可是说有误导成分'
 '我去官网下了最新的XP驱动，结果声卡装好了，有声音了，接着装显卡，装完显卡，电脑就没声音了，一共装了四次，第三次才发现显卡驱动和声卡驱动冲突， 然后在官网下了前一个显卡驱动，终于搞定了，没有冲突']
[1 1 0 0 0]


We then split the dataset for training and testing.

Next, we upload both to Amazon S3 for use later. The SageMaker Python SDK provides a helpful function for uploading to Amazon S3:

In [25]:
inputs_train = sagemaker_session.upload_data("./data/train.tsv", bucket=bucket, key_prefix=prefix)
inputs_test = sagemaker_session.upload_data("./data/test.tsv", bucket=bucket, key_prefix=prefix)

# Run training

## Training script

We use the [PyTorch-Transformers library](https://pytorch.org/hub/huggingface_pytorch-transformers), which contains PyTorch implementations and pre-trained model weights for many NLP models, including BERT.

Our training script should save model artifacts learned during training to a file path called `model_dir`, as stipulated by the SageMaker PyTorch image. Upon completion of training, model artifacts saved in `model_dir` will be uploaded to S3 by SageMaker and will become available in S3 for deployment.

We save this script in a file named `train_deploy.py`, and put the file in a directory named `code/`. The full training script can be viewed under `code/`.

In [30]:
!pygmentize code/train_deploy-cn.py

import argparse
import json
import logging
import os
import sys

import numpy as np
import pandas as pd
import torch
import torch.distributed as dist
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data import DataLoader, RandomSampler, TensorDataset
from transformers import AdamW, BertForSequenceClassification, BertTokenizer

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

MAX_LEN = 512  # this is the max length of the sentence

print("Loading BERT tokenizer...")
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese", do_lower_case=True)


def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def _get_train_data_loader(batch_size, training_dir, is_distributed):
    logger.info("Get train data loader")

    dataset = pd.read_csv(os.path.join(trainin

In [34]:
from sagemaker.pytorch import PyTorch

# place to save model artifact
output_path = f"s3://{bucket}/{prefix}"

estimator = PyTorch(
    entry_point="train_deploy-cn.py",
    source_dir="code",
    role=role,
    framework_version="1.3.1",
    py_version="py3",
    instance_count=2,  # this script only support distributed training for GPU instances.
    instance_type="ml.p3.2xlarge",
    output_path=output_path,
    hyperparameters={
        "epochs": 1,
        "num_labels": 2,
        "backend": "gloo",
    },
    disable_profiler=True, # disable debugger
)
estimator.fit({"training": inputs_train, "testing": inputs_test})

2021-09-07 07:02:50 Starting - Starting the training job...
2021-09-07 07:02:52 Starting - Launching requested ML instances......
2021-09-07 07:03:57 Starting - Preparing the instances for training.........
2021-09-07 07:05:41 Downloading - Downloading input data
2021-09-07 07:05:41 Training - Downloading the training image........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-09-07 07:07:02,170 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-09-07 07:07:02,197 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.

2021-09-07 07:07:07 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-09-07 07:07:08,722 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-09-07 07:07:08,747

# Host

After training our model, we host it on an Amazon SageMaker Endpoint. To make the endpoint load the model and serve predictions, we implement a few methods in `train_deploy.py`.

* `model_fn()`: function defined to load the saved model and return a model object that can be used for model serving. The SageMaker PyTorch model server loads our model by invoking model_fn.
* `input_fn()`: deserializes and prepares the prediction input. In this example, our request body is first serialized to JSON and then sent to model serving endpoint. Therefore, in `input_fn()`, we first deserialize the JSON-formatted request body and return the input as a `torch.tensor`, as required for BERT.
* `predict_fn()`: performs the prediction and returns the result.

To deploy our endpoint, we call `deploy()` on our PyTorch estimator object, passing in our desired number of instances and instance type:


In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

----------

We then configure the predictor to use `application/json` for the content type when sending requests to our endpoint:

In [ ]:
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

Finally, we use the returned predictor object to call the endpoint:

In [39]:
result = predictor.predict("今天心情不太好!")
print("predicted class: ", np.argmax(result, axis=1))

predicted class:  [0]


We can see the predicted class is 1 as expected because test sentence is a grammatically correct sentence. 

Before moving on, let's delete the Amazon SageMaker endpoint to avoid charges:

In [40]:
predictor.delete_endpoint()

If you want to reuse pretrained model, you can create a `PyTorchModel` from existing model artifacts. For example,
we can retrieve model artifacts we just trained. 

In [14]:
model_data = estimator.model_data
print(model_data)

s3://sagemaker-us-east-2-847380964353/sagemaker/DEMO-pytorch-bert/pytorch-training-2021-09-07-04-18-34-913/output/model.tar.gz


In [15]:
from sagemaker.pytorch.model import PyTorchModel 

pytorch_model = PyTorchModel(model_data=model_data,
                             role=role,
                             framework_version="1.3.1",
                             source_dir="code",
                             py_version="py3",
                             entry_point="train_deploy.py")

predictor = pytorch_model.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

------------!

In [16]:
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

In [17]:
result = predictor.predict("Remember to delete me when are done")
print("predicted class: ", np.argmax(result, axis=1))

predicted class:  [0]


In [18]:
# batch inference 
result = predictor.predict([
    "This is how you do batch inference", 
    "Put several sentences in a list",
    "Make sure they are shorter than 64 words"])

In [19]:
print("Predicted class: ", np.argmax(result, axis=1))

Predicted class:  [1 1 1]


In [20]:
predictor.delete_endpoint()

## Elastic Inference

Selecting the right instance type for inference requires deciding between different amounts of GPU, CPU, and memory resources, and optimizing for one of these resources on a standalone GPU instance usually leads to under-utilization of other resources. [Amazon Elastic Inference](https://aws.amazon.com/machine-learning/elastic-inference/) solves this problem by enabling us to attach the right amount of GPU-powered inference acceleration to our endpoint. In March 2020, [Elastic Inference support for PyTorch became available](https://aws.amazon.com/blogs/machine-learning/reduce-ml-inference-costs-on-amazon-sagemaker-for-pytorch-models-using-amazon-elastic-inference/) for both Amazon SageMaker and Amazon EC2.

To use Elastic Inference, we must convert our trained model to TorchScript. The location of the model artifacts is `estimator.model_data`. 

First we create a folder to save model trained model, and download the `model.tar.gz` file to local directory. 

In [42]:
%%sh -s $estimator.model_data
#mkdir model
aws s3 cp $1 model/ 
tar xvzf model/model.tar.gz --directory ./model

download: s3://sagemaker-us-east-2-847380964353/sagemaker/DEMO-pytorch-bert-cn/pytorch-training-2021-09-07-07-02-50-110/output/model.tar.gz to model/model.tar.gz
config.json
pytorch_model.bin
config.json
pytorch_model.bin


The following code converts our model into the TorchScript format:

In [43]:
import subprocess
import torch
from transformers import BertForSequenceClassification

model_torchScript = BertForSequenceClassification.from_pretrained("model/", torchscript=True)
device = "cpu"
# max length for the sentences: 64
max_len = 40

for_jit_trace_input_ids = [0] * max_len
for_jit_trace_attention_masks = [0] * max_len
for_jit_trace_input = torch.tensor([for_jit_trace_input_ids])
for_jit_trace_masks = torch.tensor([for_jit_trace_input_ids])

traced_model = torch.jit.trace(
    model_torchScript, [for_jit_trace_input.to(device), for_jit_trace_masks.to(device)]
)
torch.jit.save(traced_model, "traced_bert.pt")

subprocess.call(["tar", "-czvf", "traced_bert.tar.gz", "traced_bert.pt"])

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/transformers/models/bert/modeling_bert.py:201: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  position_ids = self.position_ids[:, past_key_values_length : seq_length + past_key_values_length]
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/transformers/modeling_utils.py:2169: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  input_tensor.shape[chunk_dim] == tensor_shape for input_tensor in input_tensors


0

Loading the TorchScript model and using it for prediction require small changes in our model loading and prediction functions. We create a new script `deploy_ei.py` that is slightly different from `train_deploy.py` script.

In [44]:
!pygmentize code/deploy_ei-cn.py

import json
import logging
import os
import sys

import torch
import torch.utils.data
import torch.utils.data.distributed
from transformers import BertTokenizer

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))

MAX_LEN = 40 # this is the max length of the sentence

print("Loading BERT tokenizer...")
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese", do_lower_case=True)


def model_fn(model_dir):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("================== objects in model_dir =====================")
    print(os.listdir(model_dir))
    loaded_model = torch.jit.load(os.path.join(model_dir, "traced_bert.pt"))
    print("================== model loaded =============================")
    
    return loaded_model.to(device)



def input_fn(request_body, request_content_type):
    """An input_fn that loads a pickled tensor"""
    if request_content_type == "applica

Next we upload TorchScript model to S3 and deploy using Elastic Inference. The accelerator_type=`ml.eia2.xlarge` parameter is how we attach the Elastic Inference accelerator to our endpoint.

In [45]:
from sagemaker.pytorch import PyTorchModel

instance_type = 'ml.m5.large'
accelerator_type = 'ml.eia2.xlarge'

# TorchScript model
tar_filename = 'traced_bert.tar.gz'

# Returns S3 bucket URL
print('Upload tarball to S3')
model_data = sagemaker_session.upload_data(path=tar_filename, bucket=bucket, key_prefix=prefix)

Upload tarball to S3


In [ ]:
import time

endpoint_name = 'bert-ei-traced-{}-{}-{}'.format(instance_type, 
                                                 accelerator_type, time.time()).replace('.', '').replace('_', '')

pytorch = PyTorchModel(
    model_data=model_data,
    role=role,
    entry_point='deploy_ei-cn.py',
    source_dir='code',
    framework_version='1.3.1',
    py_version='py3',
    sagemaker_session=sagemaker_session
)

# Function will exit before endpoint is finished creating
predictor = pytorch.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    accelerator_type=accelerator_type,
    endpoint_name=endpoint_name,
    wait=True,
)

------------

In [ ]:
predictor.serializer = sagemaker.serializers.JSONSerializer()
predictor.deserializer = sagemaker.deserializers.JSONDeserializer()

In [27]:
res = predictor.predict('测试一下啦!')
print("Predicted class:", np.argmax(res, axis=1))

Predicted class: [1]


# Cleanup

Lastly, please remember to delete the Amazon SageMaker endpoint to avoid charges:

In [ ]:
predictor.delete_endpoint()